In [ ]:
from backtesting.test import EURUSD
from backtesting.lib import TrailingStrategy
from backtesting import Backtest, Strategy
import ta
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

### TA Calculation Functions

#### SMA

In [ ]:
def sma(df, window):
    sma = ta.trend.SMAIndicator(pd.Series(df), window=window).sma_indicator()
    return sma

#### RSI

In [ ]:
def rsi(df, window=14):
    rsi = ta.momentum.RSIIndicator(pd.Series(df), window=window).rsi()
    return rsi

#### EMA

In [ ]:
def ema(df, period=200):
    ema = ta.trend.EMAIndicator(pd.Series(df), window=window).ema_indicator()
    return ema

#### MACD

In [ ]:
def macd(df):
    macd = ta.trend.MACD(pd.Series(df)).macd()
    return macd

#### Bollinger Bands

In [ ]:
def signal_h(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_hband()
def signal_l(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_lband()

#### ATR

In [ ]:
def atr(df):
    return ta.volatility.AverageTrueRange(df.High, df.Low, df.Close).average_true_range()

### Other Useful Functions

In [ ]:
def featureGeneration(data):
    close = data.Close.values

    sma10 = sma(data.Close, 10)
    sma20 = sma(data.Close, 20)
    sma50 = sma(data.Close, 50)
    sma100 = sma(data.Close, 100)
    upper = signal_h(data.Close)
    lower = signal_l(data.Close)

    # Design matrix / independent features:

    # Price-derived features
    data['X_SMA10'] = (close - sma10) / close
    data['X_SMA20'] = (close - sma20) / close
    data['X_SMA50'] = (close - sma50) / close
    data['X_SMA100'] = (close - sma100) / close

    data['X_DELTA_SMA10'] = (sma10 - sma20) / close
    data['X_DELTA_SMA20'] = (sma20 - sma50) / close
    data['X_DELTA_SMA50'] = (sma50 - sma100) / close

    # Indicator features
    data['X_MOM'] = data.Close.pct_change(periods=2)
    data['X_BB_upper'] = (upper - close) / close
    data['X_BB_lower'] = (lower - close) / close
    data['X_BB_width'] = (upper - lower) / close
    # data['X_Sentiment'] = ~data.index.to_series().between('2017-09-27', '2017-12-14')

    # Some datetime features for good measure
    data['X_day'] = data.index.dayofweek
    data['X_hour'] = data.index.hour

    data['X_ATR'] = atr(data)

    data = data.dropna().astype(float)

    return data

In [ ]:
def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    # y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    # y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    # y[y > 0] = 1
    # y[y < 0] = -1

    """
    Check the next 5 candlesticks
    If it increases for 3% first, it is +1
    If it decreases for 3% first, it is -1
    """
    for t in range(5):
        

    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

### Strategy

In [ ]:
class MLTrainOnceStrategy(Strategy):
    N_ratio = .7

    tp = .03 # 3%
    sl = .02 # 2%
    # price_delta = .004  # 0.4%

    def init(self):        
        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier(7)

        # Train the classifier in advance on the first N_TRAIN examples
        self.N_TRAIN = int(self.data.df.shape[0] * self.N_ratio)
        df = self.data.df.iloc[:self.N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        # Skip the training, in-sample data
        if len(self.data) < self.N_TRAIN:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        
        # upper = close[-1] * (1 + self.tp)
        # lower = close[-1] * (1 - self.sl)
        # upper, lower = close[-1] * (1 + np.r_[1, -1] * self.price_delta)


        if forecast == 1 and not self.position.is_long:
            tp = close[-1] + 2 * self.data['X_ATR'][-1]
            sl = close[-1] - 2 * self.data['X_ATR'][-1]
            self.buy(size=.2, tp=tp, sl=sl)
            # self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            tp = close[-1] - 2 * self.data['X_ATR'][-1]
            sl = close[-1] + 2 * self.data['X_ATR'][-1]
            self.sell(size=.2, tp=tp, sl=sl)
            # self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
        # for trade in self.trades:
        #     if current_time - trade.entry_time > pd.Timedelta('2 days'):
        #         if trade.is_long:
        #             trade.sl = max(trade.sl, low)
        #         else:
        #             trade.sl = min(trade.sl, high)




In [ ]:
class MLWalkForwardStrategy(MLTrainOnceStrategy):
    def next(self):
        # Skip the cold start period with too few values available
        if len(self.data) < self.N_TRAIN:
            return

        # Re-train the model only every 20 iterations.
        # Since 20 << N_TRAIN, we don't lose much in terms of
        # "recent training examples", but the speed-up is significant!
        if len(self.data) % 20:
            return super().next()

        # Retrain on last N_TRAIN values
        df = self.data.df[-self.N_TRAIN:]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Now that the model is fitted, 
        # proceed the same as in MLTrainOnceStrategy
        super().next()

### Data Fetching

In [ ]:
data = EURUSD.copy()
data = featureGeneration(data)

In [ ]:
data = featureGeneration(data)

X, y = get_clean_Xy(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=0)

clf = KNeighborsClassifier(7)  # Model the output based on 7 "nearest" examples
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

_ = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).plot(figsize=(15, 2), alpha=.7)
print('Classification accuracy: ', np.mean(y_test == y_pred))

### Backtesting

In [ ]:
bt = Backtest(data, MLTrainOnceStrategy, commission=.0002, margin=.05)
bt.run()
bt.plot()